In [9]:
import pandas as pd
from folium import Circle, Marker, Icon, Map

In [10]:
df_restaurants = pd.read_csv("data/restaurants.csv", index_col=0)
df_snacks = pd.read_csv("data/snacks.csv", index_col=0)
df_culture = pd.read_csv("data/culture.csv", index_col=0)
df_party = pd.read_csv("data/party.csv", index_col=0)
df_drinks = pd.read_csv("data/drinks.csv", index_col=0)
df_outdoors = pd.read_csv("data/outdoors.csv", index_col=0)
df_leisure = pd.read_csv("data/leisure.csv", index_col=0)

In [11]:
df_leisure

,name,rating,address,price_level,place,CP,latitude,longitude,geometry
0,Bowling Chamartín,4.2,"Estación de Chamartín, s / n, 28036 Madrid, Spain",1.0,bowling,28036.0,40.471123,-3.682895,"{'type': 'Point', 'coordinates': [40.4711234, ..."
1,Bowling La Ermita,4.0,"Calle Sepúlveda, 3, 28011 Madrid, Spain",1.0,bowling,28011.0,40.407067,-3.725869,"{'type': 'Point', 'coordinates': [40.4070667, ..."
2,Bowling Ilusiona Plenilunio,4.3,"C.C. Plenilunio, Calle de Aracne, 3, 28022 Mad...",1.0,bowling,28022.0,40.447502,-3.586826,"{'type': 'Point', 'coordinates': [40.4475017, ..."
3,CityPlay,4.1,"Centro Comercial Palacio de Hielo, Calle de Si...",1.0,bowling,28043.0,40.463136,-3.636201,"{'type': 'Point', 'coordinates': [40.463136, -..."
4,The City,3.9,"Centro de Ocio Heron Diversia, Av. de Bruselas...",1.0,bowling,28108.0,40.531052,-3.637870,"{'type': 'Point', 'coordinates': [40.5310524, ..."
...,...,...,...,...,...,...,...,...,...
260,Sputnik Climbing Las Rozas,5.0,"Calle París, 2, 28232 Las Rozas de Madrid, Mad...",1.0,escalada,28232.0,40.498430,-3.890783,"{'type': 'Point', 'coordinates': [40.4984304, ..."
261,Escuela de Escalada Nanuk Experience,0.0,"Sector Islas, 25, 28760 Tres Cantos, Madrid, S...",1.0,escalada,28760.0,40.592808,-3.701513,"{'type': 'Point', 'coordinates': [40.5928085, ..."
262,King Kong Climbing,4.6,"Poligono Industrial Europolis, Calle del Cabo ...",1.0,escalada,28232.0,40.498503,-3.887767,"{'type': 'Point', 'coordinates': [40.4985035, ..."
263,Tabei Adventures,5.0,"Calle de Antonio López, 254, 28041 Madrid, Spain",1.0,escalada,28041.0,40.379735,-3.696122,"{'type': 'Point', 'coordinates': [40.3797353, ..."


In [12]:
df_snacks.place.unique()

array(['cake', 'cupcakes', 'coffee', 'sweets', 'ice cream', 'juices',
       'brunch'], dtype=object)

In [13]:
map_1 = Map(location=[40.428990,-3.681209],zoom_start=15)

In [14]:
tipo = "japanese"
for i, row in df_restaurants.iterrows():
    if row["place"] == tipo:
        geom = {
            "location":[row["latitude"], row["longitude"]],
            "tooltip" : row["name"]
        }
        icon = Icon(color = "lightblue",
                        prefix = "fa",
                        icon = "cutlery",
                        icon_color = "black"
         )


    Marker(**geom,icon = icon ).add_to(map_1)
map_1

In [8]:
df_restaurants

,name,rating,address,price_level,place,CP,latitude,longitude,geometry
0,YOKALOKA,4.5,"DENTRO DEL MERCADO DE ANTÓN MARTÍN, Calle de S...",2.0,japanese,28012.0,40.411608,-3.698678,"{'type': 'Point', 'coordinates': [40.4116083, ..."
1,Konnichiwa,4.4,"Calle de Fuencarral, 98, 28004 Madrid, Spain",2.0,japanese,28004.0,40.427886,-3.701783,"{'type': 'Point', 'coordinates': [40.427886099..."
2,Nigiri Fuencarral,4.0,"Calle de Fuencarral, 91, 28004 Madrid, Spain",1.0,japanese,28004.0,40.426644,-3.701579,"{'type': 'Point', 'coordinates': [40.426644, -..."
3,Ayala Japón Restaurante japonés,4.3,"Cl. de Ayala, 67, 28001 Madrid, Spain",2.0,japanese,28001.0,40.427526,-3.676516,"{'type': 'Point', 'coordinates': [40.4275256, ..."
4,Hattori Hanzo,3.9,"Calle de Mesonero Romanos, 17, 28004 Madrid, S...",2.0,japanese,28004.0,40.420960,-3.704143,"{'type': 'Point', 'coordinates': [40.4209596, ..."
...,...,...,...,...,...,...,...,...,...
1230,Azotea del Círculo,4.2,"Azotea del Círculo de Bellas Artes de Madrid, ...",3.0,pet friendly restaurant,28004.0,40.418325,-3.696485,"{'type': 'Point', 'coordinates': [40.4183253, ..."
1231,Murillo Café,3.9,"Calle de Ruiz de Alarcón, 27, 28014 Madrid, Spain",2.0,pet friendly restaurant,28014.0,40.413059,-3.691327,"{'type': 'Point', 'coordinates': [40.4130594, ..."
1232,Juan Raro,4.2,"Calle de Miguel Servet, 7, 28012 Madrid, Spain",2.0,pet friendly restaurant,28012.0,40.406999,-3.700991,"{'type': 'Point', 'coordinates': [40.4069989, ..."
1233,Honest Greens Hortaleza,4.5,"Calle de Hortaleza, 100, 28004 Madrid, Spain",2.0,pet friendly restaurant,28004.0,40.424824,-3.697831,"{'type': 'Point', 'coordinates': [40.4248241, ..."


In [15]:
df_restaurants[df_restaurants["place"]==tipo].columns

Index(['name', 'rating', 'address', 'price_level', 'place', 'CP', 'latitude',
       'longitude', 'geometry', 'price'],
      dtype='object')

In [17]:
df_snacks.place.unique()

array(['cake', 'cupcakes', 'coffee', 'sweets', 'ice cream', 'juices',
       'brunch'], dtype=object)

In [21]:

    map_1 = Map(location=[40.428990,-3.681209],zoom_start=15)

    if category == "snacks":
        for i, row in df_snacks.iterrows():
            geom = {
                "location":[row["latitude"], row["longitude"]],
                "tooltip" : row["name"]
            }
            
        
            if row["place"] == "cake":
                icon = Icon(color = "beige",
                            prefix = "fa",
                            icon = "cutlery",
                            icon_color = "black"
                )
                
            elif row["place"] == "cupcakes":
                icon = Icon(color = "pink",
                            prefix = "fa",
                            icon = "cutlery",
                            icon_color = "black"
                )
            
            elif row["place"] == "sweets":
                icon = Icon(color = "lightblue",
                            prefix = "fa",
                            icon = "cutlery",
                            icon_color = "black"
                )
                
            elif row["place"] == "ice cream":
                icon = Icon(color = "white",
                            prefix = "fa",
                            icon = "cutlery",
                            icon_color = "black"
                )
                
            elif row["place"] == "juices":
                icon = Icon(color = "orange",
                            prefix = "fa",
                            icon = "cutlery",
                            icon_color = "black"
                )
                
            else:
                icon = Icon(color = "lightgreen",
                            prefix = "fa",
                            icon = "cutlery",
                            icon_color = "black"
                )
                
            Marker(**geom,icon = icon ).add_to(map_1)
    map_1
    

NameError: name 'category' is not defined

In [22]:
for i, row in df_snacks.iterrows():
            geom = {
                "location":[row["latitude"], row["longitude"]],
                "tooltip" : row["name"]
            }
            
        
            if row["place"] == "cake":
                icon = Icon(color = "beige",
                            prefix = "fa",
                            icon = "cutlery",
                            icon_color = "black"
                )
                
            elif row["place"] == "cupcakes":
                icon = Icon(color = "pink",
                            prefix = "fa",
                            icon = "cutlery",
                            icon_color = "black"
                )
            
            elif row["place"] == "sweets":
                icon = Icon(color = "lightblue",
                            prefix = "fa",
                            icon = "cutlery",
                            icon_color = "black"
                )
                
            elif row["place"] == "ice cream":
                icon = Icon(color = "beige",
                            prefix = "fa",
                            icon = "tag",
                            icon_color = "black"
                )
                
            elif row["place"] == "juices":
                icon = Icon(color = "pink",
                            prefix = "fa",
                            icon = "cube",
                            icon_color = "black"
                )
                
            else:
                icon = Icon(color = "rehjjd",
                            prefix = "fa",
                            icon = "car",
                            icon_color = "black"
                )
                
            Marker(**geom,icon = icon ).add_to(map_1)

<ipython-input-22-a1b76b2b439e>:44: UserWarning: color argument of Icon should be one of: {'lightred', 'purple', 'darkgreen', 'beige', 'orange', 'darkred', 'lightgreen', 'black', 'blue', 'cadetblue', 'lightblue', 'darkblue', 'darkpurple', 'pink', 'gray', 'lightgray', 'red', 'white', 'green'}.
  icon = Icon(color = "rehjjd",
